## Preprocessing

In [28]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [29]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])

In [30]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [31]:
# Look at APPLICATION_TYPE value counts for binning
application = application_df["APPLICATION_TYPE"].value_counts()
application

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [32]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
at_dict = application.to_dict()
application_types_to_replace = [k for k, v in at_dict.items() if v < 1000]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [33]:
# Look at CLASSIFICATION value counts for binning
classification = application_df["CLASSIFICATION"].value_counts()
classification

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [34]:
# You may find it helpful to look at CLASSIFICATION value counts >10
classification_filtered = classification[classification > 10]
classification_filtered

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
Name: CLASSIFICATION, dtype: int64

In [35]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
c_dict = classification.to_dict()
classifications_to_replace = [k for k, v in c_dict.items() if v < 2000]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
Other     6062
C1200     4837
Name: CLASSIFICATION, dtype: int64

In [36]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [37]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL']
X = application_df.drop(columns='IS_SUCCESSFUL')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [38]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model (Attempt1)

In [39]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_of_features = len(X_train_scaled[0])
nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=no_of_features))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 80)                3120      
                                                                 
 dense_7 (Dense)             (None, 30)                2430      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,581
Trainable params: 5,581
Non-trainable params: 0
_________________________________________________________________


In [40]:
# Compile the model
nn1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
# Train the model
fit_model = nn1.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5767 - accuracy: 0.7185
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5625 - accuracy: 0.7257
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5594 - accuracy: 0.7278
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5585 - accuracy: 0.7273
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5569 - accuracy: 0.7289
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7288
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5552 - accuracy: 0.7303
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5547 - accuracy: 0.7289
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7302
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5530 - accura

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5662 - accuracy: 0.7270 - 485ms/epoch - 2ms/step
Loss: 0.566240668296814, Accuracy: 0.7269970774650574


## Compile, Train and Evaluate the Model (Attempt2)

In [43]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_of_features = len(X_train_scaled[0])
nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=60, activation='relu', input_dim=no_of_features))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 60)                2340      
                                                                 
 dense_10 (Dense)            (None, 30)                1830      
                                                                 
 dense_11 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,201
Trainable params: 4,201
Non-trainable params: 0
_________________________________________________________________


In [44]:
# Compile the model
nn2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5777 - accuracy: 0.7161
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5631 - accuracy: 0.7249
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5603 - accuracy: 0.7266
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5593 - accuracy: 0.7292
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7278
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5560 - accuracy: 0.7299
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5562 - accuracy: 0.7296
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7297
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5552 - accuracy: 0.7299
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5540 - accura

In [46]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5693 - accuracy: 0.7273 - 489ms/epoch - 2ms/step
Loss: 0.5692771673202515, Accuracy: 0.7273469567298889


## Compile, Train and Evaluate the Model (Attempt3)

In [47]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
no_of_features = len(X_train_scaled[0])
nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=no_of_features))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=40, activation='relu'))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=20, activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 80)                3120      
                                                                 
 dense_13 (Dense)            (None, 40)                3240      
                                                                 
 dense_14 (Dense)            (None, 20)                820       
                                                                 
 dense_15 (Dense)            (None, 1)                 21        
                                                                 
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Compile the model
nn3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [49]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 4ms/step - loss: 0.5757 - accuracy: 0.7190
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5629 - accuracy: 0.7259
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5596 - accuracy: 0.7287
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7282
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5574 - accuracy: 0.7286
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5561 - accuracy: 0.7290
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5548 - accuracy: 0.7293
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5547 - accuracy: 0.7309
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7303
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5534 - accura

In [50]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5683 - accuracy: 0.7276 - 507ms/epoch - 2ms/step
Loss: 0.5683314204216003, Accuracy: 0.727580189704895


In [51]:
# Export our model to HDF5 file (Saving the best from above three attempts)
nn3.save("AlphabetSoup_Charity.h5")